In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dns_oct-jan.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'dns_oct-jan.csv'

In [ ]:
#shield ID count and anomalies
pivot_table = data.pivot_table(values = 'shieldid', index = ['dns_qdomainname', 'day'], columns = None, aggfunc = 'nunique') 
pivot_table.reset_index(inplace = True)
counts = pivot_table.groupby('dns_qdomainname')['day'].nunique()
filtered_df = pivot_table.loc[pivot_table['dns_qdomainname'].isin(counts[counts <= 7].index)] #change the number "7" for how many days you want this domain_name to show up in the dataset
anomalies = filtered_df[filtered_df.shieldid > 3].set_index('day')
data.loc[:,'anomaly'] = 0
data.loc[data.dns_qdomainname.isin(anomalies.dns_qdomainname.to_list()),'anomaly'] = 1
anomalies['shieldid_count'] = anomalies.shieldid
anomalies = anomalies.drop('shieldid', axis = 1)
anomalies = anomalies.reset_index()
pivot_table['shieldid_count'] = pivot_table.shieldid
pivot_table = pivot_table.drop('shieldid', axis = 1)
data = pd.merge(data, pivot_table, on= ['day', 'dns_qdomainname'], how = 'left')

In [ ]:
#top 1mil
path = 'https://raw.githubusercontent.com/zer0h/top-1000000-domains/master/top-1000000-domains'
top_domains = pd.read_csv(path, header=None)
top_domains.rename({0: 'dns_qdomainname'}, axis=1, inplace=True)
rank = list(range(1, 1_000_001))
top_domains['rank'] = rank
top_domains.set_index('dns_qdomainname',inplace=True)
data = data.merge(top_domains, how='left', left_on='dns_qdomainname', right_on='dns_qdomainname')

In [ ]:
#whois df columns

# load the whois data set
domains = pd.read_csv('jan26_domain_whois.csv')

# merge the two dataframes  
data = data.merge(domains, left_on='dns_qdomainname', right_on='domain', how='outer')


In [ ]:
# convert  data to csv
data.to_csv('updated_df.csv')